```shell

microk8s kubectl create deployment kapend-app --image=moseskapend/jo_ges:latest

microk8s kubectl scale deployment kapend-app --replicas=2

microk8s kubectl expose deployment kapend-app --type=NodePort --port=8000 --name=kapend-app-service

```

## Traduction En Manifestes Kubernetes 


**Manifeste de déploiement (deployment.yaml):**

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: kapend-app
spec:
  replicas: 1  # Valeur par défaut, ajustée par la commande de mise à l'échelle
  selector:
    matchLabels:
      app: kapend-app
  template:
    metadata:
      labels:
        app: kapend-app
    spec:
      containers:
      - name: kapend-app
        image: moseskapend/jo_ges:latest
        ports:
        - containerPort: 8000  # En supposant que votre conteneur expose le port 8000
```

**Explication:**

- `apiVersion` : Spécifie la version de l'API Kubernetes pour les déploiements (`apps/v1`).
- `kind` : Déclare le type de ressource comme un déploiement.
- `metadata` : Fournit des informations d'identification pour le déploiement.
  - `name` : Définit le nom du déploiement (`kapend-app`).
- `spec` : Définit l'état souhaité du déploiement.
  - `replicas` : Le nombre de réplicas (pods) à créer pour le déploiement. Il est initialement défini sur 1 (valeur par défaut) mais sera remplacé par la commande de mise à l'échelle.
  - `selector` : Un sélecteur d'étiquettes utilisé pour identifier les pods qui appartiennent à ce déploiement.
    - `matchLabels` : Spécifie les étiquettes qui doivent correspondre pour qu'un pod soit considéré comme faisant partie du déploiement.
      - `app: kapend-app` : Définit une étiquette nommée `app` avec une valeur de `kapend-app`.
  - `template` : Définit le modèle de pod qui sera utilisé pour créer des pods pour le déploiement.
    - `metadata` : Étiquettes pour les pods créés par ce déploiement.
      - `labels` : Identique au sélecteur de déploiement pour s'assurer que les pods appartiennent au déploiement.
    - `spec` : Spécification du conteneur pour les pods.
      - `containers` : Un tableau de définitions de conteneurs.
        - `name` : Le nom du conteneur (`kapend-app`).
        - `image` : L'image Docker à utiliser pour le conteneur (`dontrebootme/kapend-app:v1`).
        - `ports` : Expose un port du conteneur.
          - `containerPort` : Le port à l'intérieur du conteneur qui doit être exposé (en supposant que votre conteneur exécute un service sur le port 80).

**Manifeste de service (service.yaml):**

```yaml
apiVersion: v1
kind: Service
metadata:
  name: kapend-app-service
spec:
  selector:
    app: kapend-app
  type: NodePort
  ports:
  - port: 8000  # Port externe exposé par le service
    targetPort: 8000  # Port sur les pods vers lequel le trafic est acheminé
    nodePort: 32043  # NodePort pour accéder au service en externe (peut être n'importe quel port non utilisé)
```

**Explication:**

- `apiVersion` : Spécifie la version de l'API Kubernetes pour les services (`v1`).
- `kind` : Déclare le type de ressource comme un service.
- `metadata` : Fournit des informations d'identification pour le service.
  - `name` : Définit le nom du service (`kapend-app-service`).
- `spec` : Définit l'état souhaité du service.
  - `selector` : Un sélecteur d'étiquettes utilisé pour identifier les pods qui doivent être exposés par le service. Identique au sélecteur de déploiement.
  - `type` : Spécifie le type de service. Ici, `NodePort` est utilisé pour exposer un port sur chaque nœud du cluster.
  - `ports` : Un tableau de ports de service.
    - `port` : Le port auquel le service est exposé en externe (par exemple, accessible de l'extérieur du cluster).
    - `targetPort` : Le port sur les pods vers lequel le trafic est acheminé par le service. Correspond au port du conteneur (80 dans cet exemple).
    - `nodePort` : Le NodePort sur chaque nœud qui acheminera le trafic vers le port de service. Choisissez un numéro de port non utilisé (ici

## Ajout d'un StatefulSet MySQL

```yaml
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: mysql-statefulset
spec:
  replicas: 3 # Nombre de réplicas MySQL
  selector:
    matchLabels:
      app: mysql
  template:
    metadata:
      labels:
        app: mysql
    spec:
      containers:
      - name: mysql
        image: mysql:5.7
        ports:
        - containerPort: 3306
        volumeMounts:
        - name: mysql-persistent-storage
          mountPath: /var/lib/mysql
      volumes:
      - name: mysql-persistent-storage
        persistentVolumeClaim:
          claimName: mysql-pvc
```

Explication du fichier YAML:

* **apiVersion:** Définie la version de l'API Kubernetes utilisée pour la configuration.
* **kind:** Indique que la configuration est un StatefulSet.
* **metadata:**
    * **name:** Nom du StatefulSet.
* **spec:**
    * **replicas:** Nombre de réplicas MySQL à déployer.
    * **selector:**
        * **matchLabels:** Étiquettes utilisées pour identifier les Pods du StatefulSet.
    * **template:**
        * **metadata:**
            * **labels:** Étiquettes attribuées aux Pods du StatefulSet.
        * **spec:**
            * **containers:**
                * **name:** Nom du conteneur MySQL.
                * **image:** Image Docker contenant le serveur MySQL.
                * **ports:**
                    * **containerPort:** Port sur lequel le serveur MySQL écoute les connexions.
                * **volumeMounts:**
                    * **name:** Nom du volume persistant à monter dans le conteneur.
                    * **mountPath:** Chemin d'accès au point de montage du volume persistant dans le conteneur.
            * **volumes:**
                * **name:** Nom du volume persistant.
                * **persistentVolumeClaim:**
                    * **claimName:** Nom du PersistentVolumeClaim (PVC) à utiliser pour le volume persistant.

**Remarques:**

* Vous devez créer un PersistentVolumeClaim (PVC) avant de déployer le StatefulSet. Le PVC définit les exigences en matière de stockage pour le volume persistant.
* Vous pouvez modifier le nombre de réplicas, l'image Docker et d'autres paramètres selon vos besoins.
* Assurez-vous que le serveur MySQL est configuré pour la réplication en cluster si vous souhaitez utiliser plusieurs réplicas.

**Déploiement du StatefulSet MySQL:**

Une fois que vous avez créé le fichier YAML du StatefulSet, vous pouvez le déployer dans votre cluster Kubernetes à l'aide de la commande suivante:

```
kubectl apply -f mysql-statefulset.yaml
```

**Vérification du déploiement:**

Vous pouvez vérifier le déploiement du StatefulSet MySQL à l'aide des commandes suivantes:

```
kubectl get pods
kubectl get services
```

**Accès au serveur MySQL:**

Pour accéder au serveur MySQL à partir d'un autre Pod dans le cluster, vous pouvez utiliser le nom de service du StatefulSet:

```
mysql -h mysql-statefulset.default.svc.cluster.local -u root -p
```


## Ajout d'un Service mysql

```yaml

apiVersion: v1
kind: Service
metadata:
  name: mysql-nodeport-service
spec:
  type: NodePort
  selector:
    app: mysql
  ports:
    - port: 3306
      targetPort: 3306
      nodePort: 30036
```